FITTING MODELS

In [162]:
from sklearn import linear_model
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet

In [120]:
df = pd.read_csv('clean_data.csv')

In [121]:
def eval(y_hat, y):
#     print("Mean absolute error: %.2f" % np.mean(np.absolute(y_hat - y)))
#     print("Residual sum of squares (MSE): %.2f" % np.mean((y_hat - y) ** 2))
#     print( f'Normalized sum of squares error: {round(100*np.mean((y_hat - y) ** 2) / (np.mean(y ** 2)), 2)}%' )
#     print("R2-score: %.2f" % r2_score(y_hat, y) )
    return [round(np.mean(np.absolute(y_hat - y)), 2), round(np.mean((y_hat - y) ** 2),2), 
            round(np.mean((y_hat - y) ** 2) / (np.mean(y ** 2)),2), 
            round(r2_score(y_hat, y),2)]

In [122]:
#Train-test split
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]

metrics = {}

#MODEL 1: Simple linear regression
print('=====================')
print('Simple linear regression')
print('=====================')

#a) Data preparation
X_train = train['GrLivArea'].values.reshape(-1,1)
y_train = train['SalePrice']
X_test = test['GrLivArea'].values.reshape(-1,1)
y_test = test['SalePrice']

#b) Fitting the model
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
print ('Coefficients in simple linear regression: ', [round(num,2) for num in regr.coef_])
print ('Intercept in simple linear regression: ', round(regr.intercept_,2))

#c) Drawing a graph
# plt.scatter(train.GrLivArea, train.SalePrice,  color='blue')
# plt.plot(train.GrLivArea, regr.coef_*train.GrLivArea + regr.intercept_, '-r')
# plt.title('GrLivArea vs SalePrice')
# plt.xlabel("GrLivArea")
# plt.ylabel("SalePrice")

#d) Prediction and evaluation
y_test_hat = regr.predict(X_test)
metrics['SLR'] = eval(y_test_hat, y_test )
print('Variance score: %.2f' % regr.score(X_test, y_test))

#MODEL 2: Multi-linear regression with all features
print('=====================')
print('Multi-linear regression with all features')
print('=====================')
#a) Data preparation
X_train = train.drop(columns = ['SalePrice'])
y_train = train['SalePrice']
X_test = test.drop(columns = ['SalePrice'])
y_test = test['SalePrice']

#b) Fitting the model
ml_regr = linear_model.LinearRegression()
ml_regr.fit(X_train, y_train)
print (f'20 largest coefficients in multilinear regression: {[round(num, 2) for num in sorted(ml_regr.coef_, reverse=True)[:20]]}')
print ('Intercept in multilinear regression: %.2f' % round(ml_regr.intercept_,2))

#c) Drawing a graph - not clear how to do it in this case.

#d) Prediction and evaluation
y_test_hat = ml_regr.predict(X_test)
metrics['MLR'] = eval(y_test_hat, y_test)
print('Variance score: %.2f' % ml_regr.score(X_test, y_test))

#MODEL 3: Multi-linear regression using 3 features most correlated with the SalePrice
print('=====================')
print('Multi-linear regression using 3 features most correlated with the SalePrice')
print('=====================')
#a) Data preparation
X_train = train[['GrLivArea','OverallQual', 'ExterQual']]
y_train = train['SalePrice']
X_test = test[['GrLivArea','OverallQual', 'ExterQual']]
y_test = test['SalePrice']

#b) Fitting the model
ml3_regr = linear_model.LinearRegression()
ml3_regr.fit(X_train, y_train)
print (f'Coefficients in multilinear regression:{[round(num, 2) for num in ml3_regr.coef_]}')
print ('Intercept in multilinear regression: ', round(ml3_regr.intercept_,2))

#c) Drawing a graph - not clear how to do it in this case.

#d) Prediction and evaluation
y_test_hat = ml3_regr.predict(X_test)
metrics['MLR3'] = eval(y_test_hat, y_test)

Simple linear regression
Coefficients in simple linear regression:  [107.46]
Intercept in simple linear regression:  18087.3
Variance score: 0.58
Multi-linear regression with all features
20 largest coefficients in multilinear regression: [136739.7, 126478.74, 88428.24, 84537.37, 79812.68, 62871.15, 46533.43, 41912.18, 34245.47, 28744.96, 27812.74, 24488.27, 23894.09, 22112.31, 21980.77, 19469.72, 18532.28, 18341.69, 17599.76, 17369.59]
Intercept in multilinear regression: -1799839.24
Variance score: 0.85
Multi-linear regression using 3 features most correlated with the SalePrice
Coefficients in multilinear regression:[54.56, 23793.37, 32575.56]
Intercept in multilinear regression:  -157333.64


In [123]:
#MODEL 4: k nearest neighbors regression

#a) Data preparation 
X_train = train.drop(columns = ['SalePrice'])
y_train = train['SalePrice']
X_test = test.drop(columns = ['SalePrice'])
y_test = test['SalePrice']

#d) Prediction and evaluation
for x in range(1, 10):
    y_test_hat = KNeighborsRegressor(x).fit(X_train,y_train).predict(X_test)
#     print('=====================')
#     print(f'KNN regression with k = {x}')
#     print('=====================')
    metrics[f'KNN_{x}']= eval(y_test_hat, y_test)

In [124]:
#MODEL 5: SVM regression

#a) Data preparation 
X_train = train.drop(columns = ['SalePrice'])
y_train = train['SalePrice']
X_test = test.drop(columns = ['SalePrice'])
y_test = test['SalePrice']

#d) Prediction and evaluation
for x in range(1,5):
    y_test_hat = LinearSVR(C=1, epsilon=25000*x, max_iter=100000).fit(X_train,y_train).predict(X_test)
#     print('=====================')
#     print(f'Support vector regression with epsilon = {25000*x}')
#     print('=====================')
    metrics[f'SVM_epsilon={25000*x}'] = eval(y_test_hat, y_test)

In [125]:
#MODEL 6, 7: Decision tree and Random forest

#a) Data preparation 
X_train = train.drop(columns = ['SalePrice'])
y_train = train['SalePrice']
X_test = test.drop(columns = ['SalePrice'])
y_test = test['SalePrice']

#d) Prediction and evaluation
for x in range(1,10):
    y_test_hat = DecisionTreeRegressor(max_depth = x).fit(X_train,y_train).predict(X_test)
    metrics[f'Decision_tree_max_depth={x}'] = eval(y_test_hat, y_test)
    
    y_test_hat = RandomForestRegressor(max_depth = x).fit(X_train,y_train).predict(X_test)
    metrics[f'Random_forest_max_depth={x}'] = eval(y_test_hat, y_test)

Below we will study several regularized regression model. Their common feature is that they all penalize large regression coefficients. In case of Ridge regression it is achieved by adding $\alpha\|\theta\|^2_{\ell^2}$ to the MSE. In LASSO regression one adds $\alpha\|\theta\|_{\ell^1}$ instead. Finally, in the Elastic Net approach we add a convex combination:
$\alpha\Big[r\|\theta\|^2_{\ell^2}+(1-r)\|\theta\|_{\ell^1}\Big]$.

In [126]:
#MODEL 8,9 and 10: Regularized regression models

#a) Data preparation 
X_train = train.drop(columns = ['SalePrice'])
y_train = train['SalePrice']
X_test = test.drop(columns = ['SalePrice'])
y_test = test['SalePrice']

scaler = StandardScaler()
scaler.fit(X_train)
X_train_rescaled = scaler.transform(X_train)
X_test_rescaled = scaler.transform(X_test)

In [127]:
#d) Prediction and evaluation

for x in range(1,10):

#With rescaling

    y_test_hat = Ridge(alpha= x/10).fit(X_train_rescaled, y_train).predict(X_test_rescaled)
    metrics[f'Ridge rescaled with alpha={x/10}'] = eval(y_test_hat, y_test)
    
    y_test_hat = Lasso(alpha= x/10).fit(X_train_rescaled, y_train).predict(X_test_rescaled)
    metrics[f'Lasso rescaled with alpha={x/10}'] = eval(y_test_hat, y_test)

#Without rescaling:
    
    y_test_hat = Ridge(alpha= x/10).fit(X_train, y_train).predict(X_test)
    metrics[f'Ridge not rescaled with alpha={x/10}'] = eval(y_test_hat, y_test)

    y_test_hat = Lasso(alpha= x/10).fit(X_train, y_train).predict(X_test)
    metrics[f'Lasso not rescaled with alpha={x/10}'] = eval(y_test_hat, y_test)

    
    for r in range(1,10):
        
            y_test_hat = ElasticNet(alpha= x/10, l1_ratio = r/10).fit(X_train, y_train).predict(X_test)
            metrics[f'ElasticNet not rescaled with alpha={x/10} and r={r/10}'] = eval(y_test_hat, y_test)
            
            y_test_hat = ElasticNet(alpha= x/10, l1_ratio = r/10).fit(X_train_rescaled, y_train).predict(X_test_rescaled)
            metrics[f'ElasticNet rescaled with alpha={x/10} and r={r/10}'] = eval(y_test_hat, y_test)

In [129]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df = pd.DataFrame.from_dict(metrics, orient='index', dtype=float, columns= ['MAE', 'MSE','relative MSE', 'R2'])
sorted_df = df.sort_values(by = ['R2'], ascending = False)
sorted_df.head(5)

,MAE,MSE,relative MSE,R2
Random_forest_max_depth=9,16620.82,710063043.80,0.02,0.85
ElasticNet rescaled with alpha=0.9 and r=0.9,18099.42,857423207.74,0.02,0.84
Lasso not rescaled with alpha=0.2,19636.62,903936775.33,0.02,0.84
Ridge not rescaled with alpha=0.3,18584.27,862706359.45,0.02,0.84
Lasso rescaled with alpha=0.3,19465.12,879953091.29,0.02,0.84


Now we will build a voting regressor based on the 5 top performing models.

In [137]:
rfr9 = RandomForestRegressor(max_depth = 9)
elnet = ElasticNet(alpha= 0.9, l1_ratio = 0.9)
lasso = Lasso(alpha= 0.2)
ridge = Lasso(alpha= 0.3)

voting_reg = VotingRegressor(estimators = [('rfr', rfr9),('en',elnet),('lass', lasso), ('ridg', ridge)])
voting_reg.fit(X_train, y_train)
y_test_hat = voting_reg.predict(X_test)
metrics[f'VotingRegressor'] = eval(y_test_hat, y_test)
metrics[f'VotingRegressor']

[16071.47, 741924185.74, 0.02, 0.85]

So Voting Regressor performs better than the best model! Next we will try some boosting methods.

In [159]:
ada_boost = AdaBoostRegressor( DecisionTreeRegressor(max_depth = 10), n_estimators = 400, learning_rate = 0.5 )
y_test_hat = ada_boost.fit(X_train, y_train).predict(X_test)
metrics[f'AdaBoostRegressor'] = eval(y_test_hat, y_test)
metrics[f'AdaBoostRegressor']

[15603.6, 602160155.38, 0.02, 0.88]

The choice of parameters: max_depth = 10, n_estimators = 400, learning_rate = 0.5 is somehow arbitrary. I tried 
plugging different values here; in general the larger max_depth and n_estimators, the more accurate the model. 

In [175]:
gradient_boost = GradientBoostingRegressor( max_depth = 10, n_estimators = 400, learning_rate = 0.4 )
y_test_hat = gradient_boost.fit(X_train, y_train).predict(X_test)
metrics[f'GradientBoostingRegressor'] = eval(y_test_hat, y_test)
metrics[f'GradientBoostingRegressor']

[18344.57, 712825326.74, 0.02, 0.86]

In [177]:
df = pd.DataFrame.from_dict(metrics, orient='index', dtype=float, columns= ['MAE', 'MSE','relative MSE', 'R2'])
sorted_df = df.sort_values(by = ['R2'], ascending = False)
sorted_df.head(10)

,MAE,MSE,relative MSE,R2
AdaBoostRegressor,15603.60,602160155.38,0.02,0.88
GradientBoostingRegressor,18344.57,712825326.74,0.02,0.86
VotingRegressor,16071.47,741924185.74,0.02,0.85
Random_forest_max_depth=9,16620.82,710063043.80,0.02,0.85
Ridge rescaled with alpha=0.5,19631.16,905540160.35,0.02,0.84
ElasticNet rescaled with alpha=0.1 and r=0.9,19289.38,893389364.57,0.02,0.84
ElasticNet rescaled with alpha=0.1 and r=0.8,19043.93,883811675.69,0.02,0.84
ElasticNet rescaled with alpha=0.4 and r=0.8,18192.36,859004171.61,0.02,0.84
ElasticNet rescaled with alpha=0.1 and r=0.7,18852.69,876721787.56,0.02,0.84
ElasticNet rescaled with alpha=0.4 and r=0.9,18688.72,871324407.04,0.02,0.84


We see that the ensemble/boosting techniques improve our models.